## Integración datos de AEMET con datos de claims - Modelo de coste estimado

### 1. Import y carga de datos

In [21]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
from geopy.distance import geodesic

# Rutas
claims_path = r"C:\Users\Cesc\PycharmProjects\climascan-general\data\trusted\claims\weather_claims.parquet"
aemet_path = r"C:\Users\Cesc\PycharmProjects\climascan-general\data\trusted\aemet_deltalake"
cp_base_path = r"C:\Users\Cesc\PycharmProjects\climascan-general\data\external\data"

# Claims
df_claims = pd.read_parquet(claims_path)
print("Claims:", df_claims.shape)

# AEMET
df_aemet = pd.read_parquet(aemet_path)
print("AEMET:", df_aemet.shape)
print(df_aemet.columns.tolist())
print(df_aemet.head(5))

Claims: (404130, 7)
AEMET: (2682446, 24)
['indicativo', 'fecha', 'nombre', 'provincia', 'altitud', 'tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'hrmax', 'horahrmax', 'hrmin', 'horahrmin', 'hrmedia', 'dir', 'velmedia', 'racha', 'horaracha', 'year', 'latitud', 'longitud', 'codigo_postal']
  indicativo       fecha                 nombre provincia  altitud  tmed  \
0      8293X  2025-05-30                 XÀTIVA  VALENCIA     88.0  25.5   
1      5612B  2025-05-30   LA RODA DE ANDALUCÍA   SEVILLA    410.0  26.6   
2      7250C  2025-05-30               ABANILLA    MURCIA    174.0  24.9   
3      8270X  2025-05-30                 BICORP  VALENCIA    305.0  25.5   
4       2462  2025-05-30  PUERTO DE NAVACERRADA    MADRID   1893.0  18.9   

   prec  tmin horatmin  tmax  ... horahrmin  hrmedia   dir  velmedia racha  \
0   0.0  16.9    05:00  34.1  ...     10:30     50.0    03       3.3   6.9   
1   0.0  18.9    05:30  34.3  ...    Varias     25.0    19       3.9  11.1   
2   0.0  1

Hay que arreglar tema formatos de los campos de latitud y longitud, esto quizás se debería hacer de landind a trusted.

In [22]:
def parse_coord(coord_str):
    """
    Convierte coordenadas AEMET tipo '390806N', '003123W' a float decimal.
    """
    # Separar valor numérico y letra final
    value, direction = coord_str[:-1], coord_str[-1]

    # Convertir a float
    value = float(value) / 10000  # porque '390806' = 39.0806 grados
    if direction in ["S", "W"]:
        value = -value
    return value

# Aplicar a columnas de AEMET
df_aemet["latitud"] = df_aemet["latitud"].apply(parse_coord)
df_aemet["longitud"] = df_aemet["longitud"].apply(parse_coord)

print(df_aemet[["latitud", "longitud"]].head())


   latitud  longitud
0  39.0006   -0.3123
1  37.1140   -4.4612
2  38.1140   -1.0202
3  39.0801   -0.4724
4  40.4735   -4.0038


### 2. Cargar CP y calcular centroides

Utilizamos una fuente de datos externa que contiene los archivos GeoJSON de todas la provincias españolas. Estos nos ayudará a calcular el centroide para CP.

In [11]:
# Cargar todos los geojson de provincias
files = [os.path.join(cp_base_path, f) for f in os.listdir(cp_base_path) if f.endswith(".geojson")]
gdfs = [gpd.read_file(f) for f in files]
gdf_cp = pd.concat(gdfs, ignore_index=True)

# Reproyectar a CRS métrico (UTM 30N para España peninsular)
gdf_cp = gdf_cp.to_crs(epsg=25830)

# Calcular área y centroides en metros
gdf_cp["area"] = gdf_cp.geometry.area
gdf_cp["centroid"] = gdf_cp.geometry.centroid

# Para cada CP, nos quedamos con el polígono de mayor área
gdf_cp_largest = gdf_cp.loc[gdf_cp.groupby("COD_POSTAL")["area"].idxmax()]

# Reproyectar centroides a lat/lon
gdf_cp_largest = gdf_cp_largest.set_geometry("centroid").to_crs(epsg=4326)

# Extraer lat/lon
df_cp_unique = gdf_cp_largest[["COD_POSTAL", "centroid"]].rename(columns={"COD_POSTAL": "codigo_postal"})
df_cp_unique["codigo_postal"] = df_cp_unique["codigo_postal"].astype(str).str.zfill(5)
df_cp_unique["lat"] = df_cp_unique.geometry.y
df_cp_unique["lon"] = df_cp_unique.geometry.x

# Dejar tabla limpia
df_cp_unique = df_cp_unique[["codigo_postal", "lat", "lon"]]

print("Códigos postales únicos con centroides:", df_cp_unique.shape)
display(df_cp_unique.head())

Códigos postales únicos con centroides: (10874, 3)


,codigo_postal,lat,lon
113,01001,42.849121,-2.672393
114,01002,42.853361,-2.656582
115,01003,42.845855,-2.654556
116,01004,42.844129,-2.666576
117,01005,42.843281,-2.671967


### 3. Merge Claims + Centroides CP
A continuación, añadimos los centroides calculados previamente a nuestra base de siniestros.

In [14]:
# Normalizar CP en ambos dataframes
df_claims["codigo_postal_norm"] = df_claims["codigo_postal_norm"].astype(str).str.zfill(5)
df_cp_unique["codigo_postal"] = df_cp_unique["codigo_postal"].astype(str).str.zfill(5)

# Hacemos el join
df_claims_geo = df_claims.merge(
    df_cp_unique,
    left_on="codigo_postal_norm",
    right_on="codigo_postal",
    how="left"
)

# Filtrar los siniestros que no tienen centroides válidos (CP inválidos o ficticios tipo 08000, 28000, etc.)
df_claims_geo = df_claims_geo.dropna(subset=["lat", "lon"]).copy()

print("Claims originales:", df_claims.shape)
print("Claims con centroides válidos:", df_claims_geo.shape)

# === Renombrar columnas ===
rename_map = {
     "LOB ID": "lob",
     "Fecha ocurrencia ID": "fecha_ocurrencia",
     "Estructura Unificada (Segmento Cliente Detalle) ID": "segmento_cliente_detalle",
     "Carga": "carga"
}
df_claims_geo = df_claims_geo.rename(columns=rename_map)

# === Eliminar columnas innecesarias ===
cols_to_drop = [
     "Código Postal-Población (siniestro) ID_código_postal_siniestro", "codigo_postal"
]
df_claims_geo = df_claims_geo.drop(columns=cols_to_drop, errors="ignore")

ordered_cols = ["siniestro_hash", "lob", "segmento_cliente_detalle", "fecha_ocurrencia", "codigo_postal_norm", "lat", "lon", "carga"]
df_claims_geo = df_claims_geo[ordered_cols]

# Mostrar algunas filas de ejemplo
display(df_claims_geo.head(10))

Claims originales: (404130, 7)
Claims con centroides válidos: (381892, 10)


,siniestro_hash,lob,segmento_cliente_detalle,fecha_ocurrencia,codigo_postal_norm,lat,lon,carga
0,c16fa497af721343,Household,Particulares,2017-02-12,28294,40.480418,-4.242059,71592.15
1,54e7229a9d3ff9c7,Property,Corporaciones,2017-02-10,48001,43.265557,-2.932859,2460.12
2,62bca3d0e029db05,Prop.(Non Shops),Pymes,2017-03-04,25265,41.652348,0.971093,284.21
3,23a032e8c18218e1,Condominiuns,Particulares,2017-01-20,46008,39.473295,-0.388559,9526.63
4,7a01644d2c4686b0,Household,LE Particulares,2017-03-13,03013,38.356685,-0.474923,1615.44
5,c943ead39ba1981c,Prop.(Non Shops),Pymes,2017-02-15,24001,42.598478,-5.576910,509.99
6,5d6af4ddf0e41f1e,Condominiuns,Particulares,2017-03-24,08940,41.355317,2.076312,4723.07
8,31774253873448a5,Household,LE Particulares,2017-04-28,07015,39.556861,2.604498,38123.80
9,21172a07409159ef,Household,Particulares,2017-05-24,03610,38.492162,-0.731581,5971.37
10,9efe9133ccc5130e,Household,Particulares,2017-05-31,28008,40.428208,-3.724124,568.24


Vemos que seguimos teninedo algunos CP no válidos, así que los vamos a filtrar para no considerarlos en el analisis.

In [9]:
# 3 bis. Chequeo de códigos postales en claims

# Códigos postales únicos
unique_cps = df_claims_geo["codigo_postal_norm"].unique()
print("Total CP distintos en claims:", len(unique_cps))

# Ver ejemplos de CP sospechosos (que no están en df_cp tras el merge)
df_missing_cp = df_claims_geo[df_claims_geo["lat"].isna()]

print("Número de siniestros con CP sin centroides:", df_missing_cp.shape[0])
display(df_missing_cp[["codigo_postal_norm", "Fecha ocurrencia ID", "Carga"]].head(20))

# Ver CP únicos que no se han podido mapear
missing_cps = df_missing_cp["codigo_postal_norm"].unique()
print("Códigos postales problemáticos:", missing_cps[:50])  # solo mostramos 50 primeros


Total CP distintos en claims: 8766
Número de siniestros con CP sin centroides: 22238


,codigo_postal_norm,Fecha ocurrencia ID,Carga
7,08000,2017-03-24,11003.48
19,29000,2017-11-29,4490.61
25,28800,2017-11-06,41474.93
32,08080,2018-04-02,3364.13
50,08000,2018-09-06,1506.61
51,08000,2018-09-12,1078.67
90,08080,2017-01-03,852.75
93,08080,2017-01-03,627.05
101,08000,2017-01-03,4051.86
119,28000,2017-01-03,56.17


Códigos postales problemáticos: ['08000' '29000' '28800' '08080' '28000' '30000' '46000' '29600' '12000'
 '08400' '03500' '15000' '38000' '20000' '17000' '28700' '03000' '39000'
 '07000' '03200' '03180' '46700' '03800' '43000' '44000' '04000' '11000'
 '35000' '36000' '28980' '36439' '07410' '01000' '33400' '32000' '47000'
 '36200' '25000' '49000' '31000' '08190' '15289' '20300' '24000' '48990'
 '32232' '08240' '48000' '06000' '48900']


### 4. Interpolación k-NN
Primero vamos a hacer una prueba

In [31]:
from geopy.distance import geodesic

# 1. Normalizar fechas
df_claims_geo["fecha_ocurrencia"] = pd.to_datetime(df_claims_geo["fecha_ocurrencia"])
df_aemet["fecha"] = pd.to_datetime(df_aemet["fecha"])

# 2. Selección de variables relevantes de AEMET
vars_meteo = ["tmed", "tmin", "tmax", "prec", "hrmedia", "hrmax", "hrmin", "velmedia", "racha"]
aemet_vars = ["fecha", "latitud", "longitud"] + vars_meteo
df_aemet_sel = df_aemet[aemet_vars].copy()

def interpolate_weather_for_claim(claim_row, df_aemet, k=5, max_radius_km=50):
    """
    Interpola variables meteorológicas para un siniestro individual
    usando estaciones AEMET de la misma fecha.
    Ignora valores NaN al calcular el promedio.
    """
    fecha = claim_row["fecha_ocurrencia"]
    lat, lon = claim_row["lat"], claim_row["lon"]

    # Filtrar AEMET para esa fecha
    df_day = df_aemet[df_aemet["fecha"] == fecha]
    if df_day.empty:
        return None

    # Calcular distancias
    df_day = df_day.copy()
    df_day["dist_km"] = df_day.apply(
        lambda row: geodesic((lat, lon), (row["latitud"], row["longitud"])).km,
        axis=1
    )

    # Seleccionar estaciones más cercanas en el radio
    df_near = df_day[df_day["dist_km"] <= max_radius_km].nsmallest(k, "dist_km")
    if df_near.empty:
        return None

    # Pesos = inverso de la distancia
    weights = 1 / df_near["dist_km"]
    weights /= weights.sum()

    # Interpolación ponderada ignorando NaN
    interpolated = {}
    for var in vars_meteo:
        vals = df_near[var].values
        w = weights.values

        # Filtrar NaN
        mask = ~np.isnan(vals)
        if mask.sum() > 0:
            interpolated[var] = np.average(vals[mask], weights=w[mask])
        else:
            interpolated[var] = np.nan  # si ninguna estación tiene valor

    return interpolated

# 4. Test en un subconjunto (ejemplo: 100 siniestros)
sample_claims = df_claims_geo.head(100).copy()
results = []

for _, row in sample_claims.iterrows():
    meteo = interpolate_weather_for_claim(row, df_aemet_sel, k=5, max_radius_km=50)
    if meteo:
        enriched_row = {**row.to_dict(), **meteo}
        results.append(enriched_row)

df_claims_enriched = pd.DataFrame(results)

print("Claims enriquecidos (sample):", df_claims_enriched.shape)
display(df_claims_enriched.head())


Claims enriquecidos (sample): (100, 17)


,siniestro_hash,lob,segmento_cliente_detalle,fecha_ocurrencia,codigo_postal_norm,lat,lon,carga,tmed,tmin,tmax,prec,hrmedia,hrmax,hrmin,velmedia,racha
0,c16fa497af721343,Household,Particulares,2017-02-12,28294,40.480418,-4.242059,71592.15,4.609264,2.651004,6.549803,66.784189,88.692250,92.969120,83.726338,5.865943,19.430905
1,54e7229a9d3ff9c7,Property,Corporaciones,2017-02-10,48001,43.265557,-2.932859,2460.12,9.403981,4.778555,14.035545,0.000000,51.757832,84.726989,43.046128,1.993931,9.976691
2,62bca3d0e029db05,Prop.(Non Shops),Pymes,2017-03-04,25265,41.652348,0.971093,284.21,5.803567,0.943286,10.690947,0.199729,69.582423,98.036609,51.291551,6.124903,20.635117
3,23a032e8c18218e1,Condominiuns,Particulares,2017-01-20,46008,39.473295,-0.388559,9526.63,5.046702,3.350023,6.757077,25.811758,92.550710,97.891490,85.517471,2.578821,9.085221
4,7a01644d2c4686b0,Household,LE Particulares,2017-03-13,03013,38.356685,-0.474923,1615.44,7.994618,4.783775,11.167556,28.748599,94.406035,97.713395,84.004457,3.815602,16.741205


In [30]:
# Escoger un siniestro de prueba (ejemplo: el índice 0 del sample)
test_claim = sample_claims.iloc[2]

# Filtrar AEMET en la misma fecha
df_day = df_aemet_sel[df_aemet_sel["fecha"] == test_claim["fecha_ocurrencia"]].copy()

# Calcular distancias
df_day["dist_km"] = df_day.apply(
    lambda row: geodesic((test_claim["lat"], test_claim["lon"]), (row["latitud"], row["longitud"])).km,
    axis=1
)

# Ver las 5 estaciones más cercanas
display(df_day.nsmallest(5, "dist_km")[["latitud","longitud","dist_km","tmed","prec","velmedia","racha"]])


,latitud,longitud,dist_km,tmed,prec,velmedia,racha
1110378,41.5253,1.0631,16.061254,6.0,0.0,NaN,NaN
1110061,41.4821,1.2344,28.980631,5.3,0.0,NaN,NaN
1110400,41.4052,1.1323,30.569192,6.2,0.0,NaN,NaN
1110005,41.3326,1.1834,39.691706,4.8,1.4,7.8,25.8
1110134,41.5224,0.4522,45.610884,6.6,0.0,4.2,14.7
